In [1]:
# == importando bibliotecas == 

import pandas as pd
import numpy as np
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

In [4]:
artigos = pd.read_csv('noticias-teste.csv')
artigos.rename(columns={'Textos': 'Conteudo'}, inplace=True)
artigos

,Conteudo
0,\n O pastor petista da Comunida...
1,\n O presidente Luiz Inácio Lula da...
2,\n Enquanto aumentam as tensões por...
3,\n A Confederação Nacional da Agric...
4,\n O presidente Luiz Inácio Lula da...
5,"\n O ministro da Justiça, Flávio Di..."
6,\n O presidente do Tribunal Superio...
7,\n Ao comentar sobre o estado de sa...
8,\n O presidente Luiz Inácio Lula da...
9,\n A Câmara dos Deputados restringi...


In [202]:
artigos = {'Conteudo': ["""Ao participar da abertura do 26° encontro do Foro de São Paulo, o presidente Luiz Inácio Lula da Silva criticou a “direita fascista”, pediu união da esquerda no Brasil e disse que tem orgulho de ser chamado de “comunista”. Na cerimônia, ocorrida nesta quinta-feira, 29, em Brasília, o petista se reuniu com os partidos de esquerda da América Latina. Em seu discurso, o petista admitiu que o “discurso do patriotismo e da familia” não estão alinhados com o progressismo. “Aqui no Brasil, nós enfrentamos o discurso do costume, o discurso da família, o discurso do patriotismo. Ou seja, aqui nós enfrentamos o discurso que a gente aprendeu a historicamente combater”, afirmou o presidente da República. “Eles nos acusam de comunistas, achando que nós ficamos ofendidos com isso. Nós não ficamos ofendidos. Ficaríamos ofendidos se nos chamassem de nazista, de neofascista, de terrorista. Mas de comunista, de socialista, nunca. Isso não nos ofende. Isso nos orgulha muitas vezes”, continuou o petista.

Fundado em 1990, por iniciativa do Partido dos Trabalhadores (PT) do Brasil e do Partido Comunista de Cuba, o Foro de São Paulo é um encontro dos partidos de esquerda da América Latina e tem como lema neste ano “Integração Regional para Avançar a Soberania Latino-americana e Caribenha”. Após a pandemia, o grupo retoma suas reuniões presenciais anuais. A última edição ocorreu em Caracas , capital da Venezuela, em 2019. Além de Lula, estiverem presentes a presidente do PT, Gleisi Hoffmann, a ministra da Ciência e Tecnologia e presidente do PCdoB, Luciana Santos, e o presidente da Embratur, Marcelo Freixo. Manifestantes contrários ao encontro protestaram na porta da sede do evento. Além disso, partidos de direita, como o PL, repudiaram a presença do que chamam de “ditadores”. No Twitter, a oposição levantou a hashtag #ForaForodeSP.""".replace("\n\n", " ")]}
artigos = pd.DataFrame(artigos)

In [5]:
artigos.Conteudo.iloc[0]

'\n                O pastor petista da Comunidade de Jesus em Campina Grande (PB), José Barbosa Jr., disse que “30% dos evangélicos (no Brasil) são fascistas” e propôs a criação de uma “bolha enorme de evangélicos de esquerda” para atrair a fatia restante desse público para o PT.\n                \n                A fala foi feita nesta sexta-feira (8), durante participação do pastor em um painel sobre Políticas de Direitos para Mudar o Brasil da Conferência Eleitoral do PT para 2024, que aconteceu em Brasília.\n\n                No mesmo evento, o presidente Lula (PT) defendeu que a legenda precisa se aproximar dos evangélicos e fazer o “trabalho de base” para conseguir eleger mais candidatos no próximo pleito.\n                \n                “Essa luta da ideologia a gente tem como ganhar também por entre os evangélicos. Não pense que está tudo perdido. Tem 30% dos evangélicos que são fascistas. Com esses a gente nem conversa. Agora, tem 40%, 50% que estão em disputa. Então não va

In [6]:
# == ler a base de artigos == 

#artigos = pd.read_parquet('../../dataset/processed/artigos_de_partidos/artigos_partidos.parquet')

In [7]:
# == tornar os artigos em letras minúsculas == 

artigos.Conteudo = artigos.Conteudo.str.lower()

In [8]:
# == remover artigos contendo links == 

#artigos[artigos.Conteudo.str.contains('http')].Partido.value_counts()
#artigos = artigos[~artigos.Conteudo.str.contains('http')]

In [9]:
# == todas as sentenças após o penúltimo "." a fim de eliminar ruídos que não fazem parte do artigo == 

artigos.Conteudo = artigos.Conteudo.apply(
    lambda x : ''.join(x.split('.')[:-2])
)

In [10]:
# == mascarar nome de partido == 

partidos = ['novo', 'pcb', 'pdt', 'pl', 'pp', 'psol', 'pstu', 'mdb', 'pcdob', 'psb', 'pt', 'pv', 'rede', 'união brasil']
for partido in partidos:
    artigos.Conteudo = artigos.Conteudo.str.replace(f' {partido} ', ' ')

In [11]:
# == remover todas as palavras contendo caracteres especiais ==

artigos.Conteudo = artigos.Conteudo.apply(
    lambda x : ' '.join(word for word in x.split(' ') if word.isalnum())
)

In [12]:
# == normalização por unicode e encode ascii ==

artigos.Conteudo = artigos.Conteudo.str.lower()\
                                   .str.normalize('NFKD')\
                                   .str.encode('ascii', errors='ignore')\
                                   .str.decode('utf-8')

In [13]:
# == remover pontuações e números == 

artigos.Conteudo = artigos.Conteudo.str.replace('[^a-zA-Z]', ' ')

In [14]:
# == remover double space == 

for n in range(30):
    artigos.Conteudo = artigos.Conteudo.str.replace('  ', ' ')

In [15]:
# == analisar a contagem média de palavras por partido == 

#copy = artigos.copy()
#copy['Contagem'] = copy.Conteudo.str.count(' ')
#copy.groupby(
#    by=['Vies', 'Partido'],
#).agg(
#    ContagemPalavras = ('Contagem', 'mean'), 
#    ContagemArtigos = ('Contagem', 'count')
#)

In [214]:
# == salvar o df tratado == 

artigos.to_csv('../../dataset/processed/artigos_tratados/artigo-direita4-.csv')

In [16]:
artigos.to_csv('a.csv')

In [18]:
artigos.Conteudo.iloc[0]

'o pastor petista da comunidade de jesus em campina grande jose barbosa disse que dos evangelicos sao e propos a criacao de uma enorme de evangelicos de para atrair a fatia restante desse publico para o a fala foi feita nesta durante participacao do pastor em um painel sobre politicas de direitos para mudar o brasil da conferencia eleitoral do para que aconteceu em no mesmo o presidente lula defendeu que a legenda precisa se aproximar dos evangelicos e fazer o de para conseguir eleger mais candidatos no proximo luta da ideologia a gente tem como ganhar tambem por entre os evangelicos nao pense que esta tudo perdido tem dos evangelicos que sao fascistas com esses a gente nem conversa tem que estao em disputa entao nao vamos desistir o nucleo de evangelicos do esta ativo a gente nao quer furar a bolha a gente quer criar uma bolha enorme de evangelicos de esquerda da para chegar la e rumo a vitoria em afirmou o ao comentar sobre as criticas que recebe por ser um pastor evangelico de jose 